In [3]:
import numpy as np
import pandas as pd

In [76]:
df_num = pd.read_csv("corolla_1.csv")
df_cat = pd.read_csv("corolla_2.csv")

# len(df_num)
# len(df_cat)
df_num.head()

,ID,Price,Age,KM,HP,CC,Weight
0,1,13500,23,46986,90,2000,1165
1,2,13750,23,72937,90,2000,1165
2,3,13950,24,41711,90,2000,1165
3,4,14950,26,48000,90,2000,1165
4,5,13750,30,38500,90,2000,1170


In [77]:
df_cat.head()

,ID,FuelType,TransmissionType,Doors
0,43,Petrol,Manual,2doors
1,44,Diesel,Manual,2doors
2,45,Diesel,Manual,5doors
3,46,Diesel,Manual,5doors
4,47,Diesel,Manual,5doors


In [85]:
base_table = df_num.merge(df_cat)
base_table = base_table.dropna() 
base_table.shape

(995, 10)

In [84]:
# base_table = df_num.join(df_cat.set_index("ID"), how="left", on="ID")
# base_table.dropna()
# base_table.shape
# base_table.head()

,ID,Price,Age,KM,HP,CC,Weight,FuelType,TransmissionType,Doors
0,1,13500,23,46986,90,2000,1165,Diesel,Manual,4doors
1,2,13750,23,72937,90,2000,1165,Diesel,Manual,3doors
2,3,13950,24,41711,90,2000,1165,Diesel,Manual,4doors
3,4,14950,26,48000,90,2000,1165,Diesel,Manual,3doors
4,5,13750,30,38500,90,2000,1170,Diesel,Manual,4doors


In [86]:
base_table_auto = base_table[base_table.TransmissionType == "Automatic"]
base_table_manual = base_table[base_table.TransmissionType == "Manual"]

In [87]:
# 판매가격(Price)이 차이가 있는지 통계적으로 검정하고자 한다. 이 때, p-value
from scipy.stats import levene
stat, p = levene(base_table_auto.Price, base_table_manual.Price)


In [88]:
p

0.6209924557789868

In [89]:
base_table["TransmissionType_dummy"] =  pd.get_dummies(df["TransmissionType"], drop_first=True)
base_table.head()

,ID,Price,Age,KM,HP,CC,Weight,FuelType,TransmissionType,Doors,TransmissionType_dummy
0,1,13500,23,46986,90,2000,1165,Diesel,Manual,4doors,1
1,2,13750,23,72937,90,2000,1165,Diesel,Manual,3doors,1
2,3,13950,24,41711,90,2000,1165,Diesel,Manual,4doors,1
3,4,14950,26,48000,90,2000,1165,Diesel,Manual,3doors,1
4,5,13750,30,38500,90,2000,1170,Diesel,Manual,4doors,1


In [91]:
from scipy.stats import ttest_ind
stat, p = ttest_ind(base_table["Price"], base_table["TransmissionType_dummy"], equal_var=True)

In [74]:
p

0.0

In [92]:
# 일원분산
# from scipy.stats import f_oneway
# base_table["num_doors"] = base_table.Doors.str.extract('(\d+)').astype(int)
# base_table
# stat, p = f_oneway(base_table.Price, base_table.num_doors)

In [93]:
import statsmodels.api as sm

model = sm.OLS.from_formula("Price ~ Doors", data=base_table)
result = model.fit()


In [94]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     6.653
Date:                Wed, 21 Aug 2019   Prob (F-statistic):           0.000189
Time:                        15:02:17   Log-Likelihood:                -9591.6
No. Observations:                 995   AIC:                         1.919e+04
Df Residuals:                     991   BIC:                         1.921e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        1.237e+04    265.410     

In [95]:
# 이원분산
# from statsmodels.formula.api import ols
# from statsmodels.stats.anova import anova_lm

# formula = 

In [96]:
base_table_cate = base_table.select_dtypes(include=['object'])
base_table_num = base_table.select_dtypes(exclude=['object'])

In [97]:
base_table_num.head()

,ID,Price,Age,KM,HP,CC,Weight,TransmissionType_dummy
0,1,13500,23,46986,90,2000,1165,1
1,2,13750,23,72937,90,2000,1165,1
2,3,13950,24,41711,90,2000,1165,1
3,4,14950,26,48000,90,2000,1165,1
4,5,13750,30,38500,90,2000,1170,1


In [98]:
base_table_cate.head()

,FuelType,TransmissionType,Doors
0,Diesel,Manual,4doors
1,Diesel,Manual,3doors
2,Diesel,Manual,4doors
3,Diesel,Manual,3doors
4,Diesel,Manual,4doors


In [99]:
base_y = base_table["Price"]

In [100]:
# from scipy.stats.pearsonr
# pearsonr(x1, x3)[0]
# pearsonr(price, base_table)[0]

corr = base_table.corr(method='pearson')

In [101]:
base_table_num = base_table.loc[: , ["Age", "KM", "HP", "Weight"]]

In [102]:
base_table_num.head()

,Age,KM,HP,Weight
0,23,46986,90,1165
1,23,72937,90,1165
2,24,41711,90,1165
3,26,48000,90,1165
4,30,38500,90,1170


In [103]:
base_table_cate_dummy = pd.get_dummies(base_table_cate, drop_first=True)
base_table_cate_dummy.head()

,FuelType_Diesel,FuelType_Petrol,TransmissionType_Manual,Doors_3doors,Doors_4doors,Doors_5doors
0,1,0,1,0,1,0
1,1,0,1,1,0,0
2,1,0,1,0,1,0
3,1,0,1,1,0,0
4,1,0,1,0,1,0


In [104]:
base_X = pd.concat([base_table_cate_dummy, base_table_num], axis=1)
base_X.head()

,FuelType_Diesel,FuelType_Petrol,TransmissionType_Manual,Doors_3doors,Doors_4doors,Doors_5doors,Age,KM,HP,Weight
0,1,0,1,0,1,0,23,46986,90,1165
1,1,0,1,1,0,0,23,72937,90,1165
2,1,0,1,0,1,0,24,41711,90,1165
3,1,0,1,1,0,0,26,48000,90,1165
4,1,0,1,0,1,0,30,38500,90,1170


In [105]:
# import statsmodels.formula.api as smf
import statsmodels.api as sm

model = sm.OLS(base_y, base_X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     7582.
Date:                Wed, 21 Aug 2019   Prob (F-statistic):               0.00
Time:                        15:02:42   Log-Likelihood:                -8626.7
No. Observations:                 995   AIC:                         1.727e+04
Df Residuals:                     985   BIC:                         1.732e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
FuelType_Diesel           

In [106]:
#- 모델 계수의 유의성 검정은 유의수준 5%를 기준으로 검정하시오.
# 모델에 들어간 변수들 중 유의한 변수의 개수???


In [107]:
# 차량 연식(Age) 5년, 누적주행거리(KM) 50,000 Km, 143 마력(HP), 배기량(CC) 2000cc, 무게(Weight) 1170Kg, 사용연료(FuelType) Diesel, 변속기타입(TransmissionType) Automatic, 문 개수(Doors) 4doors 차량의 
# FuelType_Diesel	FuelType_Petrol	TransmissionType_Manual	Doors_3doors	Doors_4doors	Doors_5doors	Age	KM	HP	Weigh

In [108]:
X_test = [1, 0, 0, 0, 1, 0, 5, 50000,143, 1170]

In [109]:
y_pred = result.predict(X_test)

In [110]:
y_pred

array([21813.87007116])